In [1]:
import pymorphy2

In [24]:
# объявим где хранятся исходные данные
PATH_TRAIN = 'train.csv'
PATH_TEST = 'test.csv'

# объявим куда сохраним результат
PATH_PRED = 'pred.csv'


## Из тренировочного набора собираем статистику о встречаемости слов

# создаем словарь для хранения статистики
word_stat_dict = {}
word1_stat_dict = {}
# открываем файл на чтение в режиме текста
fl = open(PATH_TRAIN, 'rt')

# считываем первую строчку - заголовок (она нам не нужна)
fl.readline()

# в цикле читаем строчки из файла
for line in fl:
    # разбиваем строчку на три строковые переменные
    Id, Sample, Prediction = line.strip().split(',')
    # строковая переменная Prediction - содержит в себе словосочетание из 2 слов, разделим их
    word1, word2 = Prediction.split(' ')
    ww1, ww2 = Sample.split(' ')
    
    # возьмем в качестве ключа 2 первые буквы, т.к. их наличие гарантировано
    for i in range(len(ww2)-1):
        key = ww2[:len(ww2)-i]
        # если такого ключа еще нет в словаре, то создадим пустой словарь для этого ключа
        if key not in word_stat_dict:
            word_stat_dict[key] = {}
            word1_stat_dict[key] = {}
        if word1 not in word1_stat_dict[key]:
            word1_stat_dict[key][word1] = {}
        if word2 not in word1_stat_dict[key][word1]:
             word1_stat_dict[key][word1][word2] = 0
        # если текущее слово еще не встречалось, то добавим его в словарь и установим счетчик этого слова в 0
        if word2 not in word_stat_dict[key]:
            word_stat_dict[key][word2] = 0

        # увеличим значение счетчика по текущему слову на 1
        word_stat_dict[key][word2] += 1
        word1_stat_dict[key][word1][word2] +=1

# закрываем файл
fl.close()

## Строим модель

# создаем словарь для хранения статистики
most_freq_dict = {}
most1_freq_dict = {}

# проходим по словарю word_stat_dict
for key in word_stat_dict:
    # для каждого ключа получаем наиболее часто встречающееся (наиболее вероятное) слово и записываем его в словарь most_freq_dict
    most_freq_dict[key] = max(word_stat_dict[key], key=word_stat_dict[key].get)
    most1_freq_dict[key] = {}
    for word1 in word1_stat_dict[key]:
        most1_freq_dict[key][word1] = max(word1_stat_dict[key][word1], key=word1_stat_dict[key][word1].get)


## Выполняем предсказание

# открываем файл на чтение в режиме текста
fl = open(PATH_TEST, 'rt')

# считываем первую строчку - заголовок (она нам не нужна)
fl.readline()

# открываем файл на запись в режиме текста
out_fl = open(PATH_PRED, 'wt')

# записываем заголовок таблицы
out_fl.write('Id,Prediction\n')

# в цикле читаем строчки из тестового файла
for line in fl:
    # разбиваем строчку на две строковые переменные
    Id, Sample = line.strip().split(',')
    # строковая переменная Sample содержит в себе полностью первое слово и кусок второго слова, разделим их
    word1, word2_chunk = Sample.split(' ')
    # вычислим ключ для заданного фрагмента второго слова
    '''
    key = word2_chunk
    if (key in word1_stat_dict) and (word1 in  word1_stat_dict[key]):
        out_fl.write('%s,%s %s\n' % (Id, word1, most1_freq_dict[key][word1]) )
    else:
        if key in most_freq_dict:
            # если ключ есть в нашем словаре, пишем в файл предсказаний: Id, первое слово, наиболее вероятное второе слово
            out_fl.write('%s,%s %s\n' % (Id, word1, most_freq_dict[key]) )
        else:
            key = word2_chunk[:2]
            # иначе пишем наиболее часто встречающееся словосочетание в целом
            if (key in word1_stat_dict) and (word1 in  word1_stat_dict[key]):
                out_fl.write('%s,%s %s\n' % (Id, word1, most1_freq_dict[key][word1]) )
            else:
                if key in most_freq_dict:
                    # если ключ есть в нашем словаре, пишем в файл предсказаний: Id, первое слово, наиболее вероятное второе слово
                    out_fl.write('%s,%s %s\n' % (Id, word1, most_freq_dict[key]) )
                else:
                    out_fl.write('%s,%s\n' % (Id, 'что она') )
    '''
    flg = 0
    for i in range(len(word2_chunk)-1):
        key = word2_chunk[:len(word2_chunk)-i]
        if (key in word1_stat_dict) and (word1 in  most1_freq_dict[key]):
            out_fl.write('%s,%s %s\n' % (Id, word1, most1_freq_dict[key][word1]) )
            flg = 1
            break
        else:
            if key in most_freq_dict:
                # если ключ есть в нашем словаре, пишем в файл предсказаний: Id, первое слово, наиболее вероятное второе слово
                out_fl.write('%s,%s %s\n' % (Id, word1, most_freq_dict[key]) )
                flg = 1
                break
    if flg == 0:
        out_fl.write('%s,%s\n' % (Id, 'что она') )
# закрываем файлы
fl.close()
out_fl.close()

In [30]:
# объявим где хранятся исходные данные
PATH_TRAIN = 'train.csv'
PATH_TEST = 'test.csv'

# объявим куда сохраним результат
PATH_PRED = 'pred.csv'


## Из тренировочного набора собираем статистику о встречаемости слов

# создаем словарь для хранения статистики
word_stat_dict = {}
word1_stat_dict = {}
# открываем файл на чтение в режиме текста
fl = open(PATH_TRAIN, 'rt')

# считываем первую строчку - заголовок (она нам не нужна)
fl.readline()

# в цикле читаем строчки из файла
for line in fl:
    # разбиваем строчку на три строковые переменные
    Id, Sample, Prediction = line.strip().split(',')
    # строковая переменная Prediction - содержит в себе словосочетание из 2 слов, разделим их
    word1, word2 = Prediction.split(' ')
    ww1, ww2 = Sample.split(' ')
    
    # возьмем в качестве ключа 2 первые буквы, т.к. их наличие гарантировано
    for i in range(len(ww2)-1):
        key = ww2[:len(ww2)-i]
        # если такого ключа еще нет в словаре, то создадим пустой словарь для этого ключа
        if key not in word_stat_dict:
            word_stat_dict[key] = {}
            word1_stat_dict[key] = {}
        if word1 not in word1_stat_dict[key]:
            word1_stat_dict[key][word1] = {}
        if word2 not in word1_stat_dict[key][word1]:
             word1_stat_dict[key][word1][word2] = 0
        # если текущее слово еще не встречалось, то добавим его в словарь и установим счетчик этого слова в 0
        if word2 not in word_stat_dict[key]:
            word_stat_dict[key][word2] = 0

        # увеличим значение счетчика по текущему слову на 1
        word_stat_dict[key][word2] += 1
        word1_stat_dict[key][word1][word2] +=1

# закрываем файл
fl.close()

## Строим модель

# создаем словарь для хранения статистики
most_freq_dict = {}
most1_freq_dict = {}

# проходим по словарю word_stat_dict
for key in word_stat_dict:
    # для каждого ключа получаем наиболее часто встречающееся (наиболее вероятное) слово и записываем его в словарь most_freq_dict
    most_freq_dict[key] = max(word_stat_dict[key], key=word_stat_dict[key].get)
    most1_freq_dict[key] = {}
    for word1 in word1_stat_dict[key]:
        most1_freq_dict[key][word1] = max(word1_stat_dict[key][word1], key=word1_stat_dict[key][word1].get)


## Выполняем предсказание

# открываем файл на чтение в режиме текста
fl = open(PATH_TEST, 'rt')

# считываем первую строчку - заголовок (она нам не нужна)
fl.readline()

# открываем файл на запись в режиме текста
out_fl = open(PATH_PRED, 'wt')

# записываем заголовок таблицы
out_fl.write('Id,Prediction\n')

# в цикле читаем строчки из тестового файла
for line in fl:
    # разбиваем строчку на две строковые переменные
    Id, Sample = line.strip().split(',')
    # строковая переменная Sample содержит в себе полностью первое слово и кусок второго слова, разделим их
    word1, word2_chunk = Sample.split(' ')
    # вычислим ключ для заданного фрагмента второго слова
    '''
    key = word2_chunk
    if (key in word1_stat_dict) and (word1 in  word1_stat_dict[key]):
        out_fl.write('%s,%s %s\n' % (Id, word1, most1_freq_dict[key][word1]) )
    else:
        if key in most_freq_dict:
            # если ключ есть в нашем словаре, пишем в файл предсказаний: Id, первое слово, наиболее вероятное второе слово
            out_fl.write('%s,%s %s\n' % (Id, word1, most_freq_dict[key]) )
        else:
            key = word2_chunk[:2]
            # иначе пишем наиболее часто встречающееся словосочетание в целом
            if (key in word1_stat_dict) and (word1 in  word1_stat_dict[key]):
                out_fl.write('%s,%s %s\n' % (Id, word1, most1_freq_dict[key][word1]) )
            else:
                if key in most_freq_dict:
                    # если ключ есть в нашем словаре, пишем в файл предсказаний: Id, первое слово, наиболее вероятное второе слово
                    out_fl.write('%s,%s %s\n' % (Id, word1, most_freq_dict[key]) )
                else:
                    out_fl.write('%s,%s\n' % (Id, 'что она') )
    '''
    flg = 0
    for i in range(len(word2_chunk)-1):
        key = word2_chunk[:len(word2_chunk)-i]
        if (key in word1_stat_dict) and (word1 in  most1_freq_dict[key]):
            if morph.parse(most1_freq_dict[key][word1])[0].tag.POS == 'VERB':
                try:
                    out_fl.write('%s,%s %s\n' % (Id, word1, morph.parse(most1_freq_dict[key][word1])[0].inflect({morph.parse(word1)[0].tag.number})[0]) )
                except:
                    out_fl.write('%s,%s %s\n' % (Id, word1, most1_freq_dict[key][word1]) )
            else:
                try:
                    out_fl.write('%s,%s %s\n' % (Id, word1, morph.parse(most1_freq_dict[key][word1])[0].inflect({morph.parse(word1)[0].tag.number,morph.parse(word1)[0].tag.case})[0]) )
                except:
                    out_fl.write('%s,%s %s\n' % (Id, word1, most1_freq_dict[key][word1]) )
            flg = 1
            break
        else:
            if key in most_freq_dict:
                # если ключ есть в нашем словаре, пишем в файл предсказаний: Id, первое слово, наиболее вероятное второе слово
                if morph.parse(most_freq_dict[key])[0].tag.POS == 'VERB':
                    try:
                        out_fl.write('%s,%s %s\n' % (Id, word1, morph.parse(most_freq_dict[key])[0].inflect({morph.parse(word1)[0].tag.number})[0]) )
                    except:
                        out_fl.write('%s,%s %s\n' % (Id, word1, most_freq_dict[key]) )
                else:
                    try:
                        out_fl.write('%s,%s %s\n' % (Id, word1, morph.parse(most_freq_dict[key])[0].inflect({morph.parse(word1)[0].tag.number,morph.parse(word1)[0].tag.case})[0]) )
                    except:
                        out_fl.write('%s,%s %s\n' % (Id, word1, most_freq_dict[key]) )
                flg = 1
                break
    if flg == 0:
        out_fl.write('%s,%s\n' % (Id, 'что она') )
# закрываем файлы
fl.close()
out_fl.close()